In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.contrib.layers import fully_connected, batch_norm
from tensorflow.examples.tutorials.mnist import input_data
from datetime import datetime
import os
import sys
import matplotlib
matplotlib.use('Agg')
from plotting import *

In [31]:
x = tf.Variable([[1,2,3],[4,5,6]])
y = tf.argmax(x, axis=1)
y = tf.multinomial(tf.log([[10., 10.,20],[3,3,50]]), 1)
with tf.Session() as sess:
    x.initializer.run()
    result = sess.run(y)
print(result)

[[2]
 [2]]


In [32]:
# np.zeros((10,1), dtype=np.int32)

In [17]:
np.random.choice(2, size=[10,2], p = [0.98, 0.02])

array([[0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [2]:
# Loading MNIST data
mnist = input_data.read_data_sets("/tmp/data/")

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [3]:
n_s = 1
n_inputs = 28*28 - n_s
# encoders
n_hidden1 = 500
n_hidden2 = 20 # codings
n_hidden3 = 500
n_hidden4 = 20
# decoders
n_hidden5 = 500
n_hidden6 = 20
n_hidden7 = 500
# n_hidden8 = 20
# final output can take a random sample from the posterior
n_outputs = n_inputs + n_s

In [4]:
tf.reset_default_graph()

with tf.contrib.framework.arg_scope(
        [fully_connected],
        activation_fn = tf.nn.elu,
        weights_initializer = tf.contrib.layers.variance_scaling_initializer()):
    X = tf.placeholder(tf.float32, shape = [None, n_inputs])
    s = tf.placeholder(tf.float32, shape = [None, n_s])
    X_full = tf.concat([X,s], axis=1)
    y = tf.placeholder(tf.int32, shape = [None, 1])
    hidden1 = fully_connected(tf.concat([X, s], axis=1), n_hidden1)
    hidden2_mean = fully_connected(hidden1, n_hidden2, activation_fn = None)
    hidden2_gamma = fully_connected(hidden1, n_hidden2, activation_fn = None)
    hidden2_sigma = tf.exp(0.5 * hidden2_gamma)
    noise1 = tf.random_normal(tf.shape(hidden2_sigma), dtype=tf.float32)
    hidden2 = hidden2_mean + hidden2_sigma * noise1
    hidden3 = fully_connected(tf.concat([hidden2, tf.cast(y, tf.float32)], axis=1), n_hidden3)
    hidden4_mean = fully_connected(hidden3, n_hidden4, activation_fn = None)
    hidden4_gamma = fully_connected(hidden3, n_hidden4, activation_fn = None)
    hidden4_sigma = tf.exp(0.5 * hidden4_gamma)
    hidden4_softmax_mean = fully_connected(hidden3, 10, activation_fn = tf.nn.softmax)
    noise2 = tf.random_normal(tf.shape(hidden4_sigma), dtype=tf.float32)
    hidden4 = hidden4_mean + hidden4_sigma * noise2
    hidden5 = fully_connected(tf.concat([hidden4, tf.cast(y, tf.float32)], axis=1 ), n_hidden5)
    hidden6_mean = fully_connected(hidden5, n_hidden6, activation_fn = None)
    hidden6_gamma = fully_connected(hidden5, n_hidden6, activation_fn = None)
    hidden6_sigma = tf.exp(0.5 * hidden6_gamma)
    noise3 = tf.random_normal(tf.shape(hidden6_sigma), dtype=tf.float32)
    hidden6 = hidden6_mean + hidden6_sigma * noise3
    hidden7 = fully_connected(tf.concat([hidden6, s], axis=1), n_hidden7,
                             activation_fn = tf.nn.tanh)
    hidden8 = fully_connected(hidden7, n_outputs, activation_fn = None)
    outputs = tf.sigmoid(hidden8)

In [5]:
# expected lower bound
kl_z2 = 0.5 * tf.reduce_sum(
                tf.exp(hidden4_gamma)
                + tf.square(hidden4_mean)
                - 1
                - hidden4_gamma
                )

kl_z1 = 0.5 * (tf.reduce_sum(
                (1 / (1e-10 + tf.exp(hidden6_gamma))) * tf.exp(hidden2_gamma)
                - 1
                + hidden6_gamma
                - hidden2_gamma
                ) + tf.einsum('ij,ji -> i',
                    (hidden6_mean-hidden2_mean) * (1 / (1e-10 + tf.exp(hidden6_gamma))),
                    tf.transpose((hidden6_mean-hidden2_mean))))

# foo = tf.einsum('ij,ji -> i',
#                     (hidden6_mean - hidden2_mean) * (1 / (eps + tf.exp(hidden6_gamma))),
#                     tf.transpose((hidden6_mean-hidden2_mean)))

indices = tf.range(tf.shape(y)[0])
indices = tf.concat([indices[:, tf.newaxis], y], axis=1)
eps = 1e-10
log_q_y_z1 = tf.reduce_sum(tf.log(eps + tf.gather_nd(hidden4_softmax_mean, indices)))

# Bernoulli likelihood
reconstruction_loss = -(tf.reduce_sum(X_full * tf.log(outputs)
                        + (1 - X_full) * tf.log(1 - outputs)))

alpha = 10
cost = kl_z2 + kl_z1 + reconstruction_loss + alpha * log_q_y_z1

learning_rate = 0.0001
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(cost)

In [6]:
init = tf.global_variables_initializer()

In [10]:
# Training
n_epochs = 2
batch_size = 100
n_digits = 60
    
with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        n_batches = mnist.train.num_examples // batch_size
        for iteration in range(n_batches):
            X_batch, y_batch = mnist.train.next_batch(batch_size)
            sess.run(training_op, feed_dict={X: X_batch[:,:-n_s],
                                    s: X_batch[:,-n_s:],
                                    y: y_batch[:,np.newaxis]})
            kl_z2_val, kl_z1_val, log_q_y_z1_val, reconstruction_loss_val, loss_val = sess.run([
                    kl_z2,
                    kl_z1,
                    log_q_y_z1,
                    reconstruction_loss,
                    cost],
                    feed_dict={X: X_batch[:,:-n_s],
                            s: X_batch[:,-n_s:],
                            y: y_batch[:,np.newaxis]})
            print("\r{}".format(epoch), "Train total loss:", loss_val,
             "\tReconstruction loss:", reconstruction_loss_val,
              "\tKL-z1:", kl_z1_val,
              "\tKL-z2:", kl_z2_val,
              "\tlog_q(y|z1):", log_q_y_z1_val)
            print('')
            
#             print(tf.reduce_sum(tf.log(tf.gather_nd(hidden4_softmax_mean, indices))).eval(feed_dict={X: X_batch[:,:-n_s],
#                         s: X_batch[:,-n_s:],
#                         y: y_batch[:,np.newaxis]}))
#             print(tf.log(outputs).eval(feed_dict={X: X_batch[:,:-n_s],
#             s: X_batch[:,-n_s:],
#             y: y_batch[:,np.newaxis]}))

0 Train total loss: 1.75709e+13 	Reconstruction loss: 68141.3 	KL-z1: 1.75709e+13 	KL-z2: 16709.4 	log_q(y|z1): -551.242

0 Train total loss: 2.92196e+11 	Reconstruction loss: 67953.6 	KL-z1: 2.92196e+11 	KL-z2: 13018.8 	log_q(y|z1): -573.57

0 Train total loss: 4.28363e+12 	Reconstruction loss: 67590.5 	KL-z1: 4.28363e+12 	KL-z2: 16758.1 	log_q(y|z1): -589.93

0 Train total loss: 3.87694e+13 	Reconstruction loss: 67461.8 	KL-z1: 3.87694e+13 	KL-z2: 14013.9 	log_q(y|z1): -567.476

0 Train total loss: 1.9611e+13 	Reconstruction loss: 67369.3 	KL-z1: 1.9611e+13 	KL-z2: 15938.2 	log_q(y|z1): -646.743

0 Train total loss: 1.71585e+12 	Reconstruction loss: 66998.9 	KL-z1: 1.71585e+12 	KL-z2: 11423.6 	log_q(y|z1): -580.82

0 Train total loss: 6.81216e+11 	Reconstruction loss: 66730.5 	KL-z1: 6.81216e+11 	KL-z2: 10516.1 	log_q(y|z1): -631.664

0 Train total loss: 2.44021e+11 	Reconstruction loss: 65621.0 	KL-z1: 2.44021e+11 	KL-z2: 11874.8 	log_q(y|z1): -677.095

0 Train total loss: 6.55422e+

0 Train total loss: 1.42556e+07 	Reconstruction loss: 49786.3 	KL-z1: 1.42094e+07 	KL-z2: 6759.34 	log_q(y|z1): -1031.78

0 Train total loss: 2.62519e+07 	Reconstruction loss: 51394.2 	KL-z1: 2.62038e+07 	KL-z2: 7526.29 	log_q(y|z1): -1080.09

0 Train total loss: 3.56032e+08 	Reconstruction loss: 50041.7 	KL-z1: 3.55985e+08 	KL-z2: 6892.87 	log_q(y|z1): -994.489

0 Train total loss: 1.7968e+08 	Reconstruction loss: 47113.1 	KL-z1: 1.79637e+08 	KL-z2: 6962.29 	log_q(y|z1): -1041.69

0 Train total loss: 5.44474e+11 	Reconstruction loss: 53524.5 	KL-z1: 5.44474e+11 	KL-z2: 8447.85 	log_q(y|z1): -995.064

0 Train total loss: 5.03542e+09 	Reconstruction loss: 52613.7 	KL-z1: 5.03537e+09 	KL-z2: 8574.86 	log_q(y|z1): -1156.76

0 Train total loss: 6.29051e+09 	Reconstruction loss: 52389.5 	KL-z1: 6.29046e+09 	KL-z2: 7559.39 	log_q(y|z1): -1040.77

0 Train total loss: 3.42556e+09 	Reconstruction loss: 48965.4 	KL-z1: 3.42552e+09 	KL-z2: 7444.63 	log_q(y|z1): -1088.4

0 Train total loss: 1.9499

0 Train total loss: 9.782e+06 	Reconstruction loss: 41801.0 	KL-z1: 9.74642e+06 	KL-z2: 7152.35 	log_q(y|z1): -1337.25

0 Train total loss: 1.2753e+09 	Reconstruction loss: 41944.3 	KL-z1: 1.27526e+09 	KL-z2: 7193.01 	log_q(y|z1): -1321.33

0 Train total loss: 3.67979e+07 	Reconstruction loss: 42356.8 	KL-z1: 3.67621e+07 	KL-z2: 6693.08 	log_q(y|z1): -1327.22

0 Train total loss: 1.90571e+11 	Reconstruction loss: 41274.4 	KL-z1: 1.90571e+11 	KL-z2: 8185.22 	log_q(y|z1): -1426.74

0 Train total loss: 4.61757e+09 	Reconstruction loss: 42905.1 	KL-z1: 4.61754e+09 	KL-z2: 7533.92 	log_q(y|z1): -1496.37

0 Train total loss: 9.96396e+06 	Reconstruction loss: 39971.8 	KL-z1: 9.93058e+06 	KL-z2: 7582.7 	log_q(y|z1): -1418.14

0 Train total loss: 1.09248e+07 	Reconstruction loss: 41082.5 	KL-z1: 1.08901e+07 	KL-z2: 6946.16 	log_q(y|z1): -1332.76

0 Train total loss: 1.18273e+08 	Reconstruction loss: 43729.7 	KL-z1: 1.18236e+08 	KL-z2: 6684.72 	log_q(y|z1): -1369.69

0 Train total loss: 2.03879e

0 Train total loss: 1.5674e+07 	Reconstruction loss: 40200.4 	KL-z1: 1.5643e+07 	KL-z2: 6528.55 	log_q(y|z1): -1576.86

0 Train total loss: 5.06837e+06 	Reconstruction loss: 38531.6 	KL-z1: 5.03931e+06 	KL-z2: 6565.8 	log_q(y|z1): -1603.93

0 Train total loss: 1.20499e+07 	Reconstruction loss: 40633.9 	KL-z1: 1.20186e+07 	KL-z2: 6341.97 	log_q(y|z1): -1567.82

0 Train total loss: 2.3097e+07 	Reconstruction loss: 38245.3 	KL-z1: 2.30678e+07 	KL-z2: 6380.92 	log_q(y|z1): -1548.13

0 Train total loss: 2.08198e+07 	Reconstruction loss: 37122.6 	KL-z1: 2.07911e+07 	KL-z2: 6573.77 	log_q(y|z1): -1501.06

0 Train total loss: 1.98686e+08 	Reconstruction loss: 40265.8 	KL-z1: 1.98654e+08 	KL-z2: 6071.11 	log_q(y|z1): -1469.98

0 Train total loss: 2.0307e+08 	Reconstruction loss: 34205.8 	KL-z1: 2.03045e+08 	KL-z2: 6295.03 	log_q(y|z1): -1492.56

0 Train total loss: 2.12332e+07 	Reconstruction loss: 35677.3 	KL-z1: 2.12077e+07 	KL-z2: 6637.16 	log_q(y|z1): -1688.98

0 Train total loss: 3.6891e+1

0 Train total loss: 2.27855e+07 	Reconstruction loss: 32752.1 	KL-z1: 2.27638e+07 	KL-z2: 6412.2 	log_q(y|z1): -1744.25

0 Train total loss: 5.06764e+07 	Reconstruction loss: 33593.4 	KL-z1: 5.06535e+07 	KL-z2: 6724.65 	log_q(y|z1): -1747.18

0 Train total loss: 3.40179e+08 	Reconstruction loss: 35904.4 	KL-z1: 3.40154e+08 	KL-z2: 6820.94 	log_q(y|z1): -1766.44

0 Train total loss: 4.4339e+07 	Reconstruction loss: 32970.4 	KL-z1: 4.43158e+07 	KL-z2: 7192.93 	log_q(y|z1): -1700.47

0 Train total loss: 1.01578e+10 	Reconstruction loss: 34605.0 	KL-z1: 1.01577e+10 	KL-z2: 6490.68 	log_q(y|z1): -1754.14

0 Train total loss: 2.38011e+07 	Reconstruction loss: 36672.8 	KL-z1: 2.37746e+07 	KL-z2: 5748.15 	log_q(y|z1): -1592.97

0 Train total loss: 1.13148e+08 	Reconstruction loss: 33304.9 	KL-z1: 1.13125e+08 	KL-z2: 6826.67 	log_q(y|z1): -1710.78

0 Train total loss: 7.94111e+07 	Reconstruction loss: 37467.5 	KL-z1: 7.93839e+07 	KL-z2: 6962.29 	log_q(y|z1): -1719.22

0 Train total loss: 2.3724

...

1 Train total loss: 6.73273e+06 	Reconstruction loss: 27293.0 	KL-z1: 6.71811e+06 	KL-z2: 6974.88 	log_q(y|z1): -1964.91

1 Train total loss: 5.40664e+06 	Reconstruction loss: 28122.6 	KL-z1: 5.39123e+06 	KL-z2: 6336.94 	log_q(y|z1): -1905.49

1 Train total loss: 2.96933e+07 	Reconstruction loss: 28410.4 	KL-z1: 2.96782e+07 	KL-z2: 6567.39 	log_q(y|z1): -1985.46

1 Train total loss: 1.54841e+08 	Reconstruction loss: 28783.8 	KL-z1: 1.54826e+08 	KL-z2: 6633.25 	log_q(y|z1): -2075.58

1 Train total loss: 3.42472e+06 	Reconstruction loss: 25847.7 	KL-z1: 3.41186e+06 	KL-z2: 6752.78 	log_q(y|z1): -1974.5

1 Train total loss: 3.08702e+07 	Reconstruction loss: 26644.0 	KL-z1: 3.08569e+07 	KL-z2: 7514.1 	log_q(y|z1): -2091.6

1 Train total loss: 5.81799e+08 	Reconstruction loss: 25999.5 	KL-z1: 5.81786e+08 	KL-z2: 7548.31 	log_q(y|z1): -2039.8

1 Train total loss: 6.11638e+07 	Reconstruction loss: 29020.1 	KL-z1: 6.11479e+07 	KL-z2: 7078.13 	log_q(y|z1): -2015.27

1 Train total loss: 6.2963e+

1 Train total loss: 9.9554e+07 	Reconstruction loss: 27722.2 	KL-z1: 9.95405e+07 	KL-z2: 6885.66 	log_q(y|z1): -2110.4

1 Train total loss: 3.03588e+06 	Reconstruction loss: 28988.5 	KL-z1: 3.02112e+06 	KL-z2: 6670.48 	log_q(y|z1): -2089.53

1 Train total loss: 1.20962e+07 	Reconstruction loss: 28313.4 	KL-z1: 1.20811e+07 	KL-z2: 6590.96 	log_q(y|z1): -1980.33

1 Train total loss: 5.14194e+06 	Reconstruction loss: 29731.5 	KL-z1: 5.12533e+06 	KL-z2: 6884.66 	log_q(y|z1): -2000.19

1 Train total loss: 7.31241e+06 	Reconstruction loss: 27302.7 	KL-z1: 7.29899e+06 	KL-z2: 6555.9 	log_q(y|z1): -2043.44

1 Train total loss: 8.6288e+06 	Reconstruction loss: 28683.0 	KL-z1: 8.61387e+06 	KL-z2: 6363.9 	log_q(y|z1): -2011.67

1 Train total loss: 1.99932e+07 	Reconstruction loss: 29136.8 	KL-z1: 1.9978e+07 	KL-z2: 6310.22 	log_q(y|z1): -2028.45

1 Train total loss: 1.8505e+07 	Reconstruction loss: 27291.0 	KL-z1: 1.84922e+07 	KL-z2: 6468.99 	log_q(y|z1): -2089.8

1 Train total loss: 6.87949e+06 

1 Train total loss: 6.04258e+06 	Reconstruction loss: 29498.1 	KL-z1: 6.02629e+06 	KL-z2: 6322.14 	log_q(y|z1): -1952.42

1 Train total loss: 5.79146e+06 	Reconstruction loss: 29936.3 	KL-z1: 5.77513e+06 	KL-z2: 6793.28 	log_q(y|z1): -2039.98

1 Train total loss: 7.22825e+06 	Reconstruction loss: 27669.9 	KL-z1: 7.2143e+06 	KL-z2: 6929.82 	log_q(y|z1): -2065.52

1 Train total loss: 2.91269e+07 	Reconstruction loss: 27933.1 	KL-z1: 2.91126e+07 	KL-z2: 7196.31 	log_q(y|z1): -2084.84

1 Train total loss: 2.19727e+07 	Reconstruction loss: 27000.3 	KL-z1: 2.196e+07 	KL-z2: 6460.07 	log_q(y|z1): -2077.8

1 Train total loss: 2.62263e+06 	Reconstruction loss: 27459.3 	KL-z1: 2.60885e+06 	KL-z2: 6468.15 	log_q(y|z1): -2014.01

1 Train total loss: 2.2802e+06 	Reconstruction loss: 27414.5 	KL-z1: 2.2656e+06 	KL-z2: 7035.2 	log_q(y|z1): -1985.34

1 Train total loss: 1.06653e+07 	Reconstruction loss: 28315.3 	KL-z1: 1.06502e+07 	KL-z2: 6779.42 	log_q(y|z1): -2003.82

1 Train total loss: 2.69095e+06

1 Train total loss: 4.66538e+06 	Reconstruction loss: 28721.5 	KL-z1: 4.64978e+06 	KL-z2: 6741.06 	log_q(y|z1): -1986.85

1 Train total loss: 2.56682e+06 	Reconstruction loss: 26313.9 	KL-z1: 2.55488e+06 	KL-z2: 6782.15 	log_q(y|z1): -2115.57

1 Train total loss: 1.52815e+07 	Reconstruction loss: 27738.6 	KL-z1: 1.52677e+07 	KL-z2: 6707.04 	log_q(y|z1): -2068.33

1 Train total loss: 1.39968e+08 	Reconstruction loss: 28915.6 	KL-z1: 1.39952e+08 	KL-z2: 6548.29 	log_q(y|z1): -1972.29

1 Train total loss: 4.55026e+06 	Reconstruction loss: 30206.6 	KL-z1: 4.53377e+06 	KL-z2: 6477.54 	log_q(y|z1): -2019.96

1 Train total loss: 4.44547e+06 	Reconstruction loss: 30780.0 	KL-z1: 4.42952e+06 	KL-z2: 5969.38 	log_q(y|z1): -2080.05

1 Train total loss: 1.20266e+07 	Reconstruction loss: 27658.9 	KL-z1: 1.20129e+07 	KL-z2: 6303.77 	log_q(y|z1): -2019.54

1 Train total loss: 3.40368e+06 	Reconstruction loss: 29618.9 	KL-z1: 3.38824e+06 	KL-z2: 6208.65 	log_q(y|z1): -2039.02

1 Train total loss: 8.76

1 Train total loss: 1.44573e+09 	Reconstruction loss: 25464.8 	KL-z1: 1.44572e+09 	KL-z2: 6554.28 	log_q(y|z1): -1983.39

1 Train total loss: 3.20905e+06 	Reconstruction loss: 26580.6 	KL-z1: 3.19612e+06 	KL-z2: 6121.08 	log_q(y|z1): -1977.08

1 Train total loss: 4.42968e+06 	Reconstruction loss: 29611.5 	KL-z1: 4.41438e+06 	KL-z2: 6510.79 	log_q(y|z1): -2081.74

1 Train total loss: 4.65132e+06 	Reconstruction loss: 27660.7 	KL-z1: 4.63867e+06 	KL-z2: 6672.14 	log_q(y|z1): -2167.64

1 Train total loss: 9.77578e+06 	Reconstruction loss: 25657.8 	KL-z1: 9.76407e+06 	KL-z2: 6726.77 	log_q(y|z1): -2067.37

1 Train total loss: 1.11124e+06 	Reconstruction loss: 28397.1 	KL-z1: 1.09663e+06 	KL-z2: 6472.25 	log_q(y|z1): -2025.84

1 Train total loss: 1.69983e+06 	Reconstruction loss: 31721.6 	KL-z1: 1.68229e+06 	KL-z2: 6363.42 	log_q(y|z1): -2054.29

